## Problem Statement:

Company X is launching a new expense manager Android/iOS application and wants to create awareness among its target audience. They plan to offer a one-month free trial of the application to people who subscribe through the website. To achieve their marketing goal, they need to identify effective marketing channels that can reach their target audience and generate a high ROI through website sign-ups. While they have data from a previous similar campaign and have been using a heuristic model (last touch attribution) for budget allocation, they want to leverage their ML team's inputs to build a more data-driven solution that can analyze and optimize their marketing channels. 

## Solution:

Based on the given problem, the ML team suggested using Markov Chain model as it best suits attribution problems. The primary aim of the team is to find out the optimal percentage budget allocation for each marketing channel, based on the output of the Markov chain model. 

The team has access to the historical user path data along with the daily marketing spend for each of the channel.

The team proposes to build the model using the **ChannelAttribution** python package. 

## Step 1: Import the required libraries

In [1]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install ChannelAttribution

import numpy as np
import pandas as pd
from ChannelAttribution import *

Looking for attribution at path level? Try markov_model_local_api() or ask for ChannelAttributionPro at info@channelattribution.io! Visit https://channelattribution.io for more information.
Version: 2.1.3


## Step 2: Read historical campaign data (user path and daily marketing spend for the channels)

In [2]:
attribution_data = pd.read_csv('attribution_data.csv')
attribution_data['time'] = pd.to_datetime(attribution_data['time'])
attribution_data.head()

,cookie,time,interaction,conversion,conversion_value,channel
0,9AACfh9fhA7EEn0fn0hihDBfD,2018-07-21 23:25:13+00:00,impression,0,0.0,Instagram
1,CoCko0ih7EF993ABBoCnC9F3F,2018-07-13 22:49:52+00:00,impression,0,0.0,Facebook
2,hB9o3if3A33F0Eh97EDhEk9F7,2018-07-07 22:20:16+00:00,impression,0,0.0,Facebook
3,hB9o3if3A33F0Eh97EDhEk9F7,2018-07-07 22:20:48+00:00,impression,0,0.0,Facebook
4,99E9ohDDhnkiCDofo7k73nii0,2018-07-29 09:23:04+00:00,impression,0,0.0,Paid Search


In [3]:
daily_budget = pd.read_csv('daily_budget.csv')
daily_budget.head()

,day,channel,impressions,cost
0,2018-07-01,Facebook,7576,26.5160
1,2018-07-01,Instagram,3350,13.4000
2,2018-07-01,Online Display,3769,16.9605
3,2018-07-01,Online Video,2364,11.8200
4,2018-07-01,Paid Search,4992,27.4560


## Step 3: Data Pre-processing

**Creating the path order for when users interact with different channels (this is equivalent to SQL window functions)**

In [4]:
attribution_data['path_order'] = attribution_data.sort_values(['time']).groupby(['cookie']).cumcount() + 1
attribution_data.head()

,cookie,time,interaction,conversion,conversion_value,channel,path_order
0,9AACfh9fhA7EEn0fn0hihDBfD,2018-07-21 23:25:13+00:00,impression,0,0.0,Instagram,1
1,CoCko0ih7EF993ABBoCnC9F3F,2018-07-13 22:49:52+00:00,impression,0,0.0,Facebook,1
2,hB9o3if3A33F0Eh97EDhEk9F7,2018-07-07 22:20:16+00:00,impression,0,0.0,Facebook,1
3,hB9o3if3A33F0Eh97EDhEk9F7,2018-07-07 22:20:48+00:00,impression,0,0.0,Facebook,2
4,99E9ohDDhnkiCDofo7k73nii0,2018-07-29 09:23:04+00:00,impression,0,0.0,Paid Search,1


**Aggregating the channels a user interacted with into a single row**

In [5]:
attribution_data_paths = attribution_data.groupby('cookie')['channel'].agg(lambda x: x.tolist()).reset_index()
attribution_data_paths = attribution_data_paths.rename(columns={'channel': 'path'})
attribution_data_paths.head()

,cookie,path
0,00000FkCnDfDDf0iC97iC703B,"[Instagram, Online Display, Online Display, On..."
1,0000nACkD9nFkBBDECD3ki00E,"[Paid Search, Paid Search, Paid Search, Paid S..."
2,0003EfE37E93D0BC03iBhBBhF,"[Paid Search, Paid Search, Paid Search, Paid S..."
3,00073CFE3FoFCn70fBhB3kfon,[Instagram]
4,00079hhBkDF3k3kDkiFi9EFAD,[Paid Search]


**Aggregating conversions and their value by cookie ID**

In [6]:
attribution_data_conversions = attribution_data.groupby('cookie', as_index=False).agg({'conversion': 'sum', 'conversion_value': 'sum'})
attribution_data_conversions.head()

,cookie,conversion,conversion_value
0,00000FkCnDfDDf0iC97iC703B,0,0.0
1,0000nACkD9nFkBBDECD3ki00E,0,0.0
2,0003EfE37E93D0BC03iBhBBhF,0,0.0
3,00073CFE3FoFCn70fBhB3kfon,0,0.0
4,00079hhBkDF3k3kDkiFi9EFAD,0,0.0
...,...,...,...
240103,ooooE0hkAFBkED90ChDDiBFAf,0,0.0
240104,ooooEBE0o0D97ACAAAnDoi3F0,0,0.0
240105,ooooEiB0CCoEf9fiiC90Dfhfk,0,0.0
240106,ooooiBh70D3k3BfAhDFfii9h7,0,0.0


**Concatinating path data with that of conversion**

In [7]:
attribution_data_final = pd.merge(attribution_data_paths, attribution_data_conversions, how='left', on='cookie')
attribution_data_final.head()

,cookie,path,conversion,conversion_value
0,00000FkCnDfDDf0iC97iC703B,"[Instagram, Online Display, Online Display, On...",0,0.0
1,0000nACkD9nFkBBDECD3ki00E,"[Paid Search, Paid Search, Paid Search, Paid S...",0,0.0
2,0003EfE37E93D0BC03iBhBBhF,"[Paid Search, Paid Search, Paid Search, Paid S...",0,0.0
3,00073CFE3FoFCn70fBhB3kfon,[Instagram],0,0.0
4,00079hhBkDF3k3kDkiFi9EFAD,[Paid Search],0,0.0


In [8]:
print('Total conversions: {}'.format(sum(attribution_data.conversion)))
print('Total conversion rate: {}%'.format(round(sum(attribution_data.conversion) / len(attribution_data)*100)))
print('Total value of conversions: ${}'.format(round(sum(attribution_data.conversion_value))))
print('Average conversion value: ${}'.format(round(sum(attribution_data.conversion_value) / sum(attribution_data.conversion))))

Total conversions: 19613
Total conversion rate: 3%
Total value of conversions: $122529
Average conversion value: $6


**We will create a variable ‘path’ in the specific format required by the Attribution Model package where the ordered channels a user interacts with are delimited by'>'**

In [9]:
def listToString(df):  
    str1 = ""  
    for i in df['path']:  
        str1 += i + ' > '    
    return str1[:-3]

attribution_data_final['path'] = attribution_data_final.apply(listToString, axis=1)
attribution_data_final.head()

,cookie,path,conversion,conversion_value
0,00000FkCnDfDDf0iC97iC703B,Instagram > Online Display > Online Display > ...,0,0.0
1,0000nACkD9nFkBBDECD3ki00E,Paid Search > Paid Search > Paid Search > Paid...,0,0.0
2,0003EfE37E93D0BC03iBhBBhF,Paid Search > Paid Search > Paid Search > Paid...,0,0.0
3,00073CFE3FoFCn70fBhB3kfon,Instagram,0,0.0
4,00079hhBkDF3k3kDkiFi9EFAD,Paid Search,0,0.0


**Removing the user's cookie and grouping by the path to see the how many times a specific combination of channels led to a conversion or null outcome**


In [10]:
attribution_data_final.drop(columns = 'cookie', inplace = True)
attribution_data_final['null'] = np.where(attribution_data_final['conversion'] == 0,1,0)

attribution_data_final = attribution_data_final.groupby(['path'], as_index = False).sum()
attribution_data_final.rename(columns={"conversion": "total_conversions", "null": "total_null", "conversion_value": "total_conversion_value"}, inplace = True)
attribution_data_final.sample(10)

,path,total_conversions,total_conversion_value,total_null
7572,Online Display > Facebook > Paid Search > Face...,0,0.0,1
6653,Instagram > Instagram > Instagram > Instagram ...,0,0.0,1
6539,Instagram > Instagram > Facebook > Paid Search...,0,0.0,1
3820,Facebook > Instagram > Paid Search > Online Di...,0,0.0,1
1217,Facebook > Facebook > Facebook > Instagram > I...,0,0.0,1
11994,Paid Search > Online Video > Online Video > On...,0,0.0,1
3481,Facebook > Instagram > Instagram > Facebook > ...,0,0.0,1
5148,Instagram > Facebook > Facebook > Facebook > F...,1,7.5,0
6046,Instagram > Facebook > Online Video > Instagra...,0,0.0,1
827,Facebook > Facebook > Facebook > Facebook > In...,0,0.0,2


## Step 4: Building the Attribution Models (Heuristic for comparison and Markov models for calculations) and saving the results in dataframe

In [11]:
heuristic_models = heuristic_models(attribution_data_final,"path","total_conversions",var_value="total_conversion_value")

In [12]:
markov_model = markov_model(attribution_data_final, "path", "total_conversions", var_value="total_conversion_value")

Number of simulations: 100000 - Convergence reached: 0.85% < 5.00%
Percentage of simulated paths that successfully end before maximum number of steps (456) is reached: 99.99%


In [13]:
all_attribution_models_result = pd.merge(heuristic_models,markov_model,on="channel_name",how="inner")

In [14]:
all_attribution_models_result.head()

,channel_name,first_touch_conversions,first_touch_value,last_touch_conversions,last_touch_value,linear_touch_conversions,linear_touch_value,total_conversions,total_conversion_value
0,Facebook,5934.0,37019.0,6416.0,40079.0,6019.842023,37585.717058,5289.103801,33027.652909
1,Instagram,2637.0,16484.0,2774.0,17298.0,2597.664524,16227.477911,4468.380797,27891.425807
2,Paid Search,5028.0,31400.5,4301.0,26835.0,4788.638906,29890.918469,3957.708706,24710.647815
3,Online Display,2275.0,14126.5,2371.0,14791.5,2221.945317,13805.361295,2395.295285,14977.770007
4,Online Video,3739.0,23499.0,3751.0,23525.5,3984.909230,25019.525267,3502.511411,21921.503463


In [15]:
all_attribution_models_conversion_result = all_attribution_models_result[["channel_name","first_touch_conversions","last_touch_conversions",\
"linear_touch_conversions","total_conversions"]]
all_attribution_models_conversion_result.columns = ["channel_name","first_touch","last_touch","linear_touch","markov_model"]

In [20]:
all_attribution_models_conversion_value_result = all_attribution_models_result[["channel_name","first_touch_value","last_touch_value",\
"linear_touch_value","total_conversion_value"]]
all_attribution_models_conversion_value_result.columns = ["channel_name","first_touch","last_touch","linear_touch","markov_model"]

all_attribution_models_conversion_value_result = pd.melt(all_attribution_models_conversion_value_result, id_vars="channel_name")

## Step 5: Calculating the Optimal Budget for the Marketing Channels

In [22]:
daily_budget_agg = daily_budget.drop(['day', 'impressions'], axis=1).groupby('channel', as_index=False).sum('cost')

In [23]:
daily_budget_agg.columns = ['channel_name', 'cost']

In [24]:
daily_budget_agg.head()

,channel_name,cost
0,Facebook,1481.6785
1,Instagram,641.1590
2,Online Display,554.9370
3,Online Video,991.8230
4,Paid Search,1187.1415


In [28]:
roas_data = pd.merge(all_attribution_models_conversion_result, daily_budget_agg)

In [29]:
roas_data.head()

,channel_name,first_touch,last_touch,linear_touch,markov_model,cost
0,Facebook,5934.0,6416.0,6019.842023,5289.103801,1481.6785
1,Instagram,2637.0,2774.0,2597.664524,4468.380797,641.1590
2,Paid Search,5028.0,4301.0,4788.638906,3957.708706,1187.1415
3,Online Display,2275.0,2371.0,2221.945317,2395.295285,554.9370
4,Online Video,3739.0,3751.0,3984.909230,3502.511411,991.8230


In [30]:
roas_data['channel_weight'] = (roas_data['markov_model'])/(sum(roas_data['markov_model']))

In [31]:
roas_data['cost_weight'] = (roas_data['cost'])/(sum(roas_data['cost']))

In [32]:
roas_data['roas'] = (roas_data['channel_weight'])/(roas_data['cost_weight'])

In [46]:
roas_data['optimal_budget'] = (roas_data['cost'])*(roas_data['roas'])

In [55]:
roas_data['optimal_budget_percentage'] = (round((roas_data['optimal_budget'])/(sum(roas_data['optimal_budget'])),2))*100

In [56]:
roas_data.head()

,channel_name,first_touch,last_touch,linear_touch,markov_model,cost,channel_weight,cost_weight,roas,optimal_budget,optimal_budget_percentage
0,Facebook,5934.0,6416.0,6019.842023,5289.103801,1481.6785,0.269673,0.305077,0.883952,1309.733172,27.0
1,Instagram,2637.0,2774.0,2597.664524,4468.380797,641.1590,0.227828,0.132014,1.725779,1106.498714,23.0
2,Paid Search,5028.0,4301.0,4788.638906,3957.708706,1187.1415,0.201790,0.244432,0.825548,980.041718,20.0
3,Online Display,2275.0,2371.0,2221.945317,2395.295285,554.9370,0.122128,0.114261,1.068848,593.143529,12.0
4,Online Video,3739.0,3751.0,3984.909230,3502.511411,991.8230,0.178581,0.204216,0.874472,867.321866,18.0


**From the above output the team realized that Instagram and Online Display channels, although performed well in driving the conversions, were under-budgeted. They need to be given a higher share of the marketing budget, which, in this case is 23% and 12% for Instagram and Online Display respectively.**